In [ ]:
import tensorflow as tf

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [61]:
train = pd.read_csv('D:/Prog/ML/Large dataset/interesting data sets/boston_train.csv')
test = pd.read_csv('D:/Prog/ML/Large dataset/interesting data sets/boston_test.csv')
test.head(5)

,ID,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
0,3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03
1,6,0.02985,0.0,2.18,0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21
2,8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311,15.2,396.90,19.15
3,9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311,15.2,386.63,29.93
4,10,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311,15.2,386.71,17.10


In [62]:
labels = train.iloc[:,-1:]
train = train.iloc[:,:-1]
train = train.iloc[:,1:]
test = test.iloc[:,1:]
print("Training set: {}".format(train.shape)) # 333 примера, 14 свойств
print("Testing set: {}".format(test.shape)) # 173 примера, 14 свойств

Training set: (333, 13)
Testing set: (173, 13)


In [63]:
print(train.iloc[1])

crim         0.02731
zn           0.00000
indus        7.07000
chas         0.00000
nox          0.46900
rm           6.42100
age         78.90000
dis          4.96710
rad          2.00000
tax        242.00000
ptratio     17.80000
black      396.90000
lstat        9.14000
Name: 1, dtype: float64


In [64]:
mean = train.mean(axis=0)
std = train.std(axis=0)
train = (train-mean)/std
test = (test-mean)/std
print(train.iloc[0])

crim      -0.456188
zn         0.322421
indus     -1.283699
chas      -0.252400
nox       -0.166537
rm         0.439492
age       -0.107574
dis        0.191844
rad       -0.987584
tax       -0.663065
ptratio   -1.462969
black      0.432339
lstat     -1.066167
Name: 0, dtype: float64


In [52]:
def build_model():
    model = keras.Sequential([
        keras.layers.Dense(64, activation=tf.nn.relu, input_shape=(trainshape[1],)),
        keras.layers.Dense(64,activation=tf.nn.relu),
        keras.layers.Dense(1)
    ])
    optimizer = tf.train.RMSPropOptimizer(0.001)
    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['mae'])
    return model
model =Build_model()
model.summary()

ID         250.951952
crim         3.360341
zn          10.689189
indus       11.293483
chas         0.060060
nox          0.557144
rm           6.265619
age         68.226426
dis          3.709934
rad          9.633634
tax        409.279279
ptratio     18.448048
black      359.466096
lstat       12.515435
medv        22.768769
dtype: float64

In [18]:
class PrintDot(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0: print('')
        print('.', end='')
EPOCHS = 500
history = model.fit(train_data, train_labels, epochs=EPOCHS,
                    validation_split=0.2, verbose=0,
                    callbacks=[PrintDot()])

In [20]:
import matplotlib.pyplot as plt


def plot_history(history):
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error [1000$]')
    plt.plot(history.epoch, 
             np.array(history.history['mean_absolute_error']),
             label='Train Loss')
    plt.plot(history.epoch, 
             np.array(history.history['val_mean_absolute_error']),
             label = 'Val loss')
    plt.legend()
    plt.ylim([0, 5])

plot_history(history)

8

In [ ]:
model = build_model()

# Параметр терпения - количество эпох, чтобы проверить улучшение
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                           patience=20)

history = model.fit(train_data, train_labels, epochs=EPOCHS,
                    validation_split=0.2, verbose=0,
                    callbacks=[early_stop, PrintDot()])

plot_history(history)

In [ ]:
[loss, mae] = model.evaluate(test_data, test_labels, verbose=0)

print("Testing set Mean Abs Error: ${:7.2f}".format(mae * 1000))

In [ ]:
test_predictions = model.predict(test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [1000$]')
plt.ylabel('Predictions [1000$]')
plt.axis('equal')
plt.xlim(plt.xlim())
plt.ylim(plt.ylim())
_ = plt.plot([-100, 100], [-100, 100])

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins = 50)
plt.xlabel("Prediction Error [1000$]")
_ = plt.ylabel("Count")